# Normalize a dataset using z-score and find out the Dissimilarity matrix

## Numerical data

In [1]:
import pandas as pd
import numpy as np

In [2]:
# creating an array of random integers
f1 = np.random.randint(low = 0, high = 100, size = 4)
f2 = np.random.randint(low = 30, high = 1000, size = 4)
f3 = np.random.randint(low = 50, high = 10000, size = 4)
f4 = np.random.randint(low = 100, high = 10000000, size = 4)

# creating a dataframe from the arrays
data = pd.DataFrame({"f1":f1, "f2":f2, "f3":f3, "f4":f4})

In [3]:
data

,f1,f2,f3,f4
0,46,196,3270,8709077
1,60,97,3350,5980393
2,45,960,1796,4341810
3,84,574,1127,4072911


We can consider *f1*, *f2*, *f3*, *f4* as features

In [4]:
# function to find the z-score
def z_score(data):
    data_z_score = pd.DataFrame()
    data_z_score = data.copy()
    for f in range(len(data)):
        sum = 0
        for i in range(len(data)):
            sum += data.iloc[i][f]
        mean = (sum / len(data))
        
        # mean absolute deviation
        mad = 0
        for i in range(len(data)):
            mad += abs(data.iloc[i][f] - mean)
        
        mad /= len(data)
        
        # z-score
        for i in range(len(data)):
            data_z_score.iloc[i][f] = (data.iloc[i][f] - mean) / mad
    
    return data_z_score
    
data_z_score = z_score(data)

In [5]:
data_z_score

,f1,f2,f3,f4
0,0,0,0,1
1,0,-1,1,0
2,-1,1,0,0
3,1,0,-1,-1


In [6]:
def euclidean_distance(x, y):
    distance = 0
    for i in range(len(x)):
        distance += (x[i] - y[i])**2
    
    return np.sqrt(distance)

We are required to find the **dissimilarity matrix** from the above matrix of *z-scores*

In [41]:
dissimilarity_matrix = np.zeros((4,4)) # creates a 2D numpy array of shape (4,4) having all values as 0
for f in range(len(data_z_score)):
    for i in range(len(data_z_score)):
        dissimilarity_matrix[i][f] = euclidean_distance(data_z_score.iloc[i].values, data_z_score.iloc[f].values)

print(dissimilarity_matrix)

[[0.         1.73205081 1.73205081 2.44948974]
 [1.73205081 0.         2.44948974 2.64575131]
 [1.73205081 2.44948974 0.         2.64575131]
 [2.44948974 2.64575131 2.64575131 0.        ]]


## Nominal data

In [2]:
hair_color = ['Black', 'Gray', 'Gray', "Blue"]
emp_status = ['Employed', 'Unemployed', 'Employed', 'Retired']
dress_color = ['blue', 'black', 'black', 'gray']
gender = ['Male', 'Female', 'Female', 'Male']

nominal_df = pd.DataFrame({'hair_color':hair_color, 'emp_status':emp_status,
                           'dress_color':dress_color, 'gender':gender})

In [3]:
nominal_df

,hair_color,emp_status,dress_color,gender
0,Black,Employed,blue,Male
1,Gray,Unemployed,black,Female
2,Gray,Employed,black,Female
3,Blue,Retired,gray,Male


In [30]:
def distinct(feature, nominal_df):
    distinct_count = dict()
    for i in nominal_df[feature]:
        if i in distinct_count:
            distinct_count[i] += 1
        else:
            distinct_count[i] = 1
    return distinct_count

In [35]:
def dissimilarity_matrix(nominal_df):
    p = len(nominal_df.columns)
    dissimilarity_matrix = [[0 for _ in range(len(nominal_df))] for _ in range(len(nominal_df))]
    for col in range(len(nominal_df.columns)):
        distinct_count = distinct(nominal_df.columns.values[col], nominal_df)
        for row in range(len(nominal_df)):
            dissimilarity_matrix[row][col] = (p - distinct_count[nominal_df.iloc[row][col]]) / p
            
    return dissimilarity_matrix

In [36]:
dissimilarity_matrix(nominal_df)

[[0.75, 0.5, 0.75, 0.5],
 [0.5, 0.75, 0.5, 0.5],
 [0.5, 0.5, 0.5, 0.5],
 [0.75, 0.75, 0.75, 0.5]]